In [ ]:
import numpy as np
import math
from quantopian.research import run_pipeline
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.filters import QTradableStocksUS

from quantopian.pipeline.factors import Latest
from quantopian.pipeline.data import morningstar
from quantopian.pipeline.factors import CustomFactor, SimpleMovingAverage, AverageDollarVolume, Returns, RSI
from quantopian.pipeline.classifiers.morningstar import Sector
from quantopian.pipeline.data.zacks import EarningsSurprises

import talib
import pandas as pd
import alphalens as al
import pyfolio as pf
from scipy import stats
import matplotlib.pyplot as plt
from sklearn import linear_model, decomposition, ensemble, preprocessing, isotonic, metrics

# Define our Factor

In [ ]:
bs = morningstar.balance_sheet
cfs = morningstar.cash_flow_statement
is_ = morningstar.income_statement
or_ = morningstar.operation_ratios
er = morningstar.earnings_report
v = morningstar.valuation
vr = morningstar.valuation_ratios


def make_factors():
    def PE():
        """
        Price to earnings
        """
        return vr.pe_ratio
        

# Define our universe

In [ ]:
universe = QTradableStocksUS

# Create Pipeline

In [ ]:
def make_pipeline():
    mean_close_10 = SimpleMovingAverage(
        inputs=[USEquityPricing.close],
        window_length=300
    )
    universe = QTradableStocksUS
    
    return Pipeline(columns = {'10month_mean_close':mean_close_10,
                              'longs': mean_close_10 < USEquityPricing.close.latest,
                              'shorts': mean_close_10 > USEquityPricing.close.latest})



In [ ]:
result = run_pipeline(make_pipeline(), '2015-01-01', '2016-01-01')

In [ ]:
result.head()

In [ ]:
result.tail()

In [ ]:
assets = result.index.levels[1].unique()
len(assets)

In [ ]:
pricing = get_pricing(assets, start_date = '2014-12-01', end_date = '2016-02-01', fields = 'open_price')

In [ ]:
import alphalens

factor_data = alphalens.utils.get_clean_factor_and_forward_returns(factor = result['10month_mean_close'],
                                        prices = pricing,
                                        quantiles = 4,
                                        periods=(1,5,10))

alphalens.tears.create_full_tear_sheet(factor_data)  